In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,36152
2,Huelva,Confirmados PDIA 14 días,2001
3,Huelva,Tasa PDIA 14 días,"389,92926320712434"
4,Huelva,Confirmados PDIA 7 días,1140
5,Huelva,Tasa PDIA 7 dias,"222,14860572519828"
6,Huelva,Total Confirmados,36349
7,Huelva,Curados,32955
8,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  36152.0


/tmp/ipykernel_77047/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  10047.0


/tmp/ipykernel_77047/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_77047/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_77047/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_77047/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 366 personas en los últimos 7 días 

Un positivo PCR cada 200 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,36152.0,1140.0,2001.0,513170.0,222.148606,389.929263,246.0
Huelva-Costa,21161.0,799.0,1417.0,289548.0,275.947339,489.383453,170.0
Huelva (capital),10047.0,393.0,719.0,143837.0,273.225943,499.871382,67.0
Condado-Campiña,11494.0,234.0,425.0,156231.0,149.778213,272.033079,41.0
Sierra de Huelva-Andévalo Central,3123.0,104.0,143.0,67391.0,154.323278,212.194507,33.0
Isla Cristina,2476.0,82.0,176.0,21393.0,383.302950,822.699014,30.0
Lepe,2373.0,78.0,141.0,27880.0,279.770445,505.738881,19.0
Ayamonte,1517.0,71.0,115.0,21104.0,336.429113,544.920394,16.0
Aljaraque,1315.0,73.0,112.0,21474.0,339.945981,521.560957,15.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Castaño del Robledo,9.0,2.0,2.0,228.0,877.192982,877.192982,NaN
Isla Cristina,2476.0,82.0,176.0,21393.0,383.302950,822.699014,30.0
Niebla,311.0,6.0,32.0,4116.0,145.772595,777.453839,1.0
Zarza-Perrunal (La),39.0,5.0,9.0,1253.0,399.042298,718.276137,3.0
Cortelazor,15.0,2.0,2.0,299.0,668.896321,668.896321,0.0
Beas,261.0,23.0,27.0,4341.0,529.831836,621.976503,7.0
Palos de la Frontera,814.0,46.0,69.0,11742.0,391.756089,587.634134,6.0
Ayamonte,1517.0,71.0,115.0,21104.0,336.429113,544.920394,16.0
Aljaraque,1315.0,73.0,112.0,21474.0,339.945981,521.560957,15.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Niebla,311.0,6.0,32.0,4116.0,145.772595,777.453839,1.0,0.187500
Aracena,565.0,6.0,18.0,8255.0,72.683222,218.049667,3.0,0.333333
Puebla de Guzmán,148.0,1.0,3.0,3092.0,32.341527,97.024580,0.0,0.333333
Palma del Condado (La),1236.0,11.0,28.0,10801.0,101.842422,259.235256,0.0,0.392857
Santa Bárbara de Casa,37.0,2.0,5.0,1043.0,191.754554,479.386385,0.0,0.400000
Minas de Riotinto,135.0,2.0,5.0,3812.0,52.465897,131.164743,1.0,0.400000
Isla Cristina,2476.0,82.0,176.0,21393.0,383.302950,822.699014,30.0,0.465909
Cumbres Mayores,56.0,1.0,2.0,1749.0,57.175529,114.351058,0.0,0.500000
San Juan del Puerto,622.0,12.0,24.0,9411.0,127.510360,255.020720,2.0,0.500000
